In [ ]:
import mlx_whisper
import os
from pathlib import Path
from datetime import datetime
from openai import OpenAI
import shutil
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set the directory paths
voice_memo_dir = Path(os.getenv('VOICE_MEMO_DIR'))
completed_dir = voice_memo_dir / 'completed'
transcription_folder = Path(os.getenv('TRANSCRIPTION_FOLDER'))


# Create the 'completed' folder if it doesn't exist
completed_dir.mkdir(exist_ok=True)

# Create the 'Memos' folder if it doesn't exist
transcription_folder.mkdir(exist_ok=True)

def markdown_memo(prompt, text):
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
    completion = client.chat.completions.create(
        model="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF",
        messages=[
            {"role": "system", "content": f"{prompt}"},
            {"role": "user", "content": f"This is the transcribed text: {text}"}
        ],
        temperature=0.2,
    )
    return completion.choices[0].message.content

titler_message = "You are a titler. You take transcribed text from a voice memo of a meeting and write a title for the meeting. The title must be less than 10 words, contain NO special characters or symbols, just words. Only return the title and no other commentary."
classifier_message = "You are a classifier and you classify transcribed text into one of these categories: 'game development', 'economics'. Return only the class which can be game development or economics."
role_message = "You are a report writer. You take transcribed text from a voice memo of a build review meeting and write a report for the team. The sections must be Summary, Issues, Suggestions, Compliments, and Next Steps. Breakdown issues into sections like UI, gameplay, art, bugs. The report must be written in Markdown format."
assistant_message = "You are a report writer. You take transcribed text from a voice memo of a meeting and write a report for the team. The sections must be Summary, Issues, Suggestions, Compliments, and Next Steps. Breakdown issues into any clear themes. The report must be written in Markdown format."

# Get all .m4a files in the directory
m4a_files = list(voice_memo_dir.glob('*.m4a'))

for speech_file in m4a_files:
    # Process the file
    text = mlx_whisper.transcribe(str(speech_file))["text"]
    
    # Get the current date and time
    formatted_datetime = datetime.now().strftime("%B %d %Y at %I_%M %p")
    
    # Generate title and classification
    title = markdown_memo(titler_message, text)
    classification = markdown_memo(classifier_message, text)
    
    # Generate report based on classification
    if classification == "game development":
        report = markdown_memo(role_message, text)
    else:
        report = markdown_memo(assistant_message, text)

    # Append the transcription to the report
    report += f"\n\n## Transcription\n\n{text}"
    
    # Write the markdown to a file
    output_file = transcription_folder / f"{title}_{formatted_datetime}.md"
    with open(output_file, "w") as f:
        f.write(report)
    
    # Move the processed file to the 'completed' folder
    shutil.move(str(speech_file), str(completed_dir / speech_file.name))

print("All .m4a files have been processed and moved to the 'completed' folder.")